In [1]:
import duckdb

# Load the CSV file into DuckDB
con = duckdb.connect()
results = con.sql("SELECT * FROM '../results/층_naive_bayes_예측결과.csv'")
# Display the first 5 rows of the results
display(results.df().head())

,주_용도_코드,주_용도_코드_명,기타_용도,Count,predicted_label,is_correct,predicted_label_name
0,17100,일반공장,비가림처마,1,01001,0,단독주택
1,03109,공공도서관,정화조점검층,1,01001,0,단독주택
2,17100,일반공장,"일반공장 (창고,물탱크실)(말소)",1,17100,1,일반공장
3,04005,제조업소,교반실,1,01001,0,단독주택
4,02003,다세대주택,빗물탱크(연면적제외),1,02003,1,다세대주택


In [10]:
# Calculate the overall accuracy
overall_accuracy = con.sql("""
    SELECT 
        SUM(Count * is_correct) / SUM(Count) AS overall_accuracy
    FROM results
""")
print("Overall Accuracy: {:.2f}%".format(overall_accuracy.df().iloc[0, 0] * 100))

# calculate the accuracy by 주_용도_코드
accuracy = con.sql("""
    SELECT 
        주_용도_코드, 
        주_용도_코드_명, 
        SUM(Count) AS total_count, 
        SUM(Count * is_correct) AS correct_count, 
        SUM(Count * is_correct) / SUM(Count) AS accuracy
    FROM results
    WHERE LENGTH("주_용도_코드") = 5 AND "주_용도_코드" ~ '^\\d+$'
    GROUP BY 주_용도_코드, 주_용도_코드_명
    HAVING SUM(Count) >= 100
    ORDER BY SUM(Count) DESC
""")
accuracy_df = (
    accuracy.df()
    .astype({"total_count": "int", "correct_count": "int"})
    .head(20)
    .sort_values(by="주_용도_코드", ascending=True)
)
accuracy_df["accuracy"] = accuracy_df["accuracy"].apply(
    lambda x: "{:.2f}%".format(x * 100)
)
display(accuracy_df)
# Save the accuracy results to a CSV file
accuracy_df.to_csv(
    "../results/층_naive_bayes_accuracy.csv", index=False, encoding="utf-8-sig"
)
accuracy.df().astype({"total_count": "int", "correct_count": "int"}).to_csv(
    "../results/층_naive_bayes_accuracy_orig.csv", index=False, encoding="utf-8-sig"
)

Overall Accuracy: 78.97%


,주_용도_코드,주_용도_코드_명,total_count,correct_count,accuracy
0,01001,단독주택,6107851,5698090,93.29%
2,01003,다가구주택,1922416,1445495,75.19%
1,02001,아파트,2866578,2673013,93.25%
17,02002,연립주택,156682,111829,71.37%
3,02003,다세대주택,1260327,1074819,85.28%
12,02005,부대시설,234099,114137,48.76%
4,03001,소매점,902487,800650,88.72%
19,03002,휴게음식점,116871,100708,86.17%
10,03999,기타제1종근린생활시설,327167,165524,50.59%
7,04001,일반음식점,563247,495787,88.02%


In [15]:
# # top and bottom 5 by accuracy
# top_5_accuracy = accuracy.df().nlargest(5, "accuracy")
# bottom_5_accuracy = accuracy.df().query("accuracy > 0").nsmallest(5, "accuracy")
# print("Top 5 Accuracy:")
# display(top_5_accuracy)
# print("Bottom 5 Accuracy:")
# display(bottom_5_accuracy)

In [24]:
# Calculate the average Count for is_correct values 0 and 1 grouped by 주_용도_코드 and 주_용도_코드_명
average_count = con.sql("""
	SELECT 
		주_용도_코드, 
		주_용도_코드_명,
		AVG(CASE WHEN is_correct = 1 THEN Count ELSE NULL END) AS avg_count_correct,
		AVG(CASE WHEN is_correct = 0 THEN Count ELSE NULL END) AS avg_count_incorrect,
	FROM results
    WHERE LENGTH("주_용도_코드") = 5 AND "주_용도_코드" ~ '^\\d+$'
	GROUP BY 주_용도_코드, 주_용도_코드_명
    HAVING SUM(Count) >= 100000
    ORDER BY 주_용도_코드
""")
average_count_df = average_count.df()
# format the average counts to 2 decimal places
average_count_df["avg_count_correct"] = average_count_df["avg_count_correct"].apply(
    lambda x: "{:.2f}".format(x)
)
average_count_df["avg_count_incorrect"] = average_count_df["avg_count_incorrect"].apply(
    lambda x: "{:.2f}".format(x)
)
display(average_count_df)
# Save the average counts to a CSV file
average_count_df.to_csv(
    "../results/층_naive_bayes_average_count.csv", index=False, encoding="utf-8-sig"
)

,주_용도_코드,주_용도_코드_명,avg_count_correct,avg_count_incorrect
0,01001,단독주택,216.30,31.33
1,01003,다가구주택,93.28,47.43
2,02001,아파트,399.61,9.10
3,02002,연립주택,55.39,8.87
4,02003,다세대주택,54.51,17.22
5,02005,부대시설,13.97,11.53
6,03001,소매점,63.01,6.97
7,03002,휴게음식점,87.04,5.90
8,03005,의원,45.11,3.83
9,03999,기타제1종근린생활시설,61.86,6.70
